In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 500)

In [4]:
!pip install s3fs

  Using cached botocore-1.22.8-py3-none-any.whl (8.1 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.23.32
    Uninstalling botocore-1.23.32:
      Successfully uninstalled botocore-1.23.32
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
boto3 1.20.16 requires botocore<1.24.0,>=1.23.16, but you have botocore 1.22.8 which is incompatible.


In [5]:
# fix for attributeerror: 'AioClientCreator' object has no attribute '_register_lazy_block_unknown_fips_pseudo_regions'
!pip install aiobotocore==2.0.1
!pip install botocore==1.22.8
!pip install aiohttp==3.7.2

# MAKE SURE TO RESTART KERNEL AFTER THIS OR FIRST pd.read_csv() WILL FAIL

In [2]:
nanpa = pd.read_csv('s3://hc-data-temp/npa_report-3.csv', dtype={'NPA_ID': str})

In [3]:
area_codes = pd.read_csv('s3://hc-data-temp/us-area-code-cities.csv', dtype={'areacode': str})

In [4]:
county_zcta = pd.read_csv('s3://hc-data-temp/county_zcta.csv', dtype={'state': str,'county': str,'zcta': str})

In [5]:
us = pd.read_csv('s3://hc-data-temp/US.txt', sep='\t', dtype={'postal_code': str})

In [6]:
zcta_county_rel_10 = pd.read_csv('s3://hc-data-temp/zcta_county_rel_10.csv', dtype={'ZCTA5': str,'STATE': str,'COUNTY': str,'GEOID': str})

In [7]:
ZiptoZcta_Crosswalk = pd.read_csv('s3://hc-data-temp/ZiptoZcta_Crosswalk_2021.csv', dtype={'ZIP_CODE': str})
ZiptoZcta_Crosswalk['ZIP_CODE'] = ZiptoZcta_Crosswalk['ZIP_CODE'].str.zfill(5)

In [8]:
all_geocodes = pd.read_csv('s3://hc-data-temp/all-geocodes-v2020.csv') #, dtype={'ANI': str}
all_geocodes.set_axis(['summary', 'state', 'county', 'county_sub', 'place', 'city', 'area'], axis=1, inplace=True)
all_geocodes = all_geocodes.astype(str)

In [9]:
tier1geoid = pd.read_csv('s3://hc-data-temp/top_geoids_feb22.csv', dtype={'Geoid': str,'State': str,'Top': str})
tier1geoid['Geoid'] = tier1geoid['Geoid'].str.zfill(5)

In [10]:
state_map = pd.read_csv('s3://hc-data-temp/state_map.csv',dtype={'state_code': str,'state': str,'code': str})
state_map['state_code'] = state_map['state_code'].str.zfill(2)

In [11]:
print(nanpa.info())
nanpa.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   NPA_ID                         800 non-null    object 
 1   type_of_code                   800 non-null    object 
 2   ASSIGNABLE                     800 non-null    object 
 3   EXPLANATION                    121 non-null    object 
 4   RESERVED                       800 non-null    object 
 5   ASSIGNED                       800 non-null    object 
 6   ASSIGNMENT_DT                  410 non-null    object 
 7   USE                            690 non-null    object 
 8   LOCATION                       457 non-null    object 
 9   COUNTRY                        446 non-null    object 
 10  IN_SERVICE                     800 non-null    object 
 11  IN_SERVICE_DT                  436 non-null    object 
 12  STATUS                         58 non-null     obj

,NPA_ID,type_of_code,ASSIGNABLE,EXPLANATION,RESERVED,ASSIGNED,ASSIGNMENT_DT,USE,LOCATION,COUNTRY,IN_SERVICE,IN_SERVICE_DT,STATUS,PLANNING_LETTERS,NOTES,OVERLAY,OVERLAY_COMPLEX,PARENT_NPA_ID,SERVICE,TIME_ZONE,AREA_SERVED,MAP,IN_JEOPARDY,RELIEF_PLANNING_IN_PROGRESS,HOME_NPA_LOCAL_CALLS,HOME_NPA_TOLL_CALLS,FOREIGN_NPA_LOCAL_CALLS,FOREIGN_NPA_TOLL_CALLS,PERM_HNPA_LOCAL_CALLS,PERM_HNPA_TOLL_CALLS,PERM_HNPA_FOREIGN_LOCAL_CALLS,DIALING_PLAN_NOTES
0,200,Easily Recognizable Code,Yes,NaN,No,No,NaN,N,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201,General Purpose Code,Yes,NaN,No,Yes,1-Jan-47,G,NJ,US,Y,1-Jan-47,NaN,277,NaN,Yes,201/551,NaN,NaN,E,NaN,277,No,No,10D,10D,1+10D,1+10D,1+10D,NaN,NaN,Calls between the 551 and 201 NPAs may be dialed as 10D.
2,202,General Purpose Code,Yes,NaN,No,Yes,1-Jan-47,G,DC,US,Y,1-Jan-47,NaN,547,NaN,Yes,202/771,NaN,NaN,E,NaN,547,No,No,7D,1+10D,10D,1+10D,10D,NaN,10D,1+10 digit permissible at each service providers discretion
3,203,General Purpose Code,Yes,NaN,No,Yes,1-Jan-47,G,CT,US,Y,1-Jan-47,NaN,388 255 217 138,NaN,Yes,203/475,NaN,NaN,E,NaN,217,No,No,10D,1+10D,10D,1+10D,1+10D,NaN,1+10D,NaN
4,204,General Purpose Code,Yes,NaN,No,Yes,1-Jan-47,G,MANITOBA,CANADA,Y,1-Jan-47,Active,435 433 419,NaN,Yes,204/431,NaN,NaN,C,NaN,435,No,Yes,10D,1+10D,10D,1+10D,NaN,NaN,NaN,NaN


In [12]:
print(area_codes.info())
area_codes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2766 entries, 0 to 2765
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   areacode  2766 non-null   object 
 1   city      2766 non-null   object 
 2   state     2766 non-null   object 
 3   country   2766 non-null   object 
 4   lat       2766 non-null   float64
 5   long      2766 non-null   float64
dtypes: float64(2), object(4)
memory usage: 129.8+ KB
None


,areacode,city,state,country,lat,long
0,201,Bayonne,New Jersey,US,40.66871,-74.11431
1,201,Bergenfield,New Jersey,US,40.92760,-73.99736
2,201,Cliffside Park,New Jersey,US,40.82149,-73.98764
3,201,Englewood,New Jersey,US,40.89288,-73.97264
4,201,Fair Lawn,New Jersey,US,40.94038,-74.13181


In [13]:
county_zcta['county'] = county_zcta['county'].str.zfill(3)
print(county_zcta.info())
county_zcta.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33139 entries, 0 to 33138
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   state         33139 non-null  object
 1   county        33139 non-null  object
 2   zcta          33139 non-null  object
 3   zcta_geoid    33139 non-null  object
 4   county_geoid  33139 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB
None


,state,county,zcta,zcta_geoid,county_geoid
0,31,039,68788,86000US68788,05000US31039
1,31,039,68716,86000US68716,05000US31039
2,31,039,68791,86000US68791,05000US31039
3,31,039,68004,86000US68004,05000US31039
4,53,069,98647,86000US98647,05000US53069


In [14]:
print(us.info())
us.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41483 entries, 0 to 41482
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  41483 non-null  object 
 1   postal_code   41483 non-null  object 
 2   place_name    41483 non-null  object 
 3   admin_name1   40972 non-null  object 
 4   admin_code1   40972 non-null  object 
 5   admin_name2   40972 non-null  object 
 6   admin_code2   40972 non-null  float64
 7   admin_name3   1 non-null      object 
 8   admin_code3   1 non-null      float64
 9   latitude      41483 non-null  float64
 10  longitude     41483 non-null  float64
 11  accuracy      40951 non-null  float64
dtypes: float64(5), object(7)
memory usage: 3.8+ MB
None


,country_code,postal_code,place_name,admin_name1,admin_code1,admin_name2,admin_code2,admin_name3,admin_code3,latitude,longitude,accuracy
0,US,99553,Akutan,Alaska,AK,Aleutians East,13.0,NaN,NaN,54.1430,-165.7854,1.0
1,US,99571,Cold Bay,Alaska,AK,Aleutians East,13.0,NaN,NaN,55.1858,-162.7211,1.0
2,US,99583,False Pass,Alaska,AK,Aleutians East,13.0,NaN,NaN,54.8542,-163.4113,1.0
3,US,99612,King Cove,Alaska,AK,Aleutians East,13.0,NaN,NaN,55.0628,-162.3056,1.0
4,US,99661,Sand Point,Alaska,AK,Aleutians East,13.0,NaN,NaN,55.3192,-160.4914,1.0


In [15]:
print(zcta_county_rel_10.info())
zcta_county_rel_10.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44410 entries, 0 to 44409
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ZCTA5          44410 non-null  object 
 1   STATE          44410 non-null  object 
 2   COUNTY         44410 non-null  object 
 3   GEOID          44410 non-null  object 
 4   POPPT          44410 non-null  int64  
 5   HUPT           44410 non-null  int64  
 6   AREAPT         44410 non-null  int64  
 7   AREALANDPT     44410 non-null  int64  
 8   ZPOP           44410 non-null  int64  
 9   ZHU            44410 non-null  int64  
 10  ZAREA          44410 non-null  int64  
 11  ZAREALAND      44410 non-null  int64  
 12  COPOP          44410 non-null  int64  
 13  COHU           44410 non-null  int64  
 14  COAREA         44410 non-null  int64  
 15  COAREALAND     44410 non-null  int64  
 16  ZPOPPCT        44410 non-null  float64
 17  ZHUPCT         44410 non-null  float64
 18  ZAREAP

,ZCTA5,STATE,COUNTY,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,ZAREA,ZAREALAND,COPOP,COHU,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
0,00601,72,001,72001,18465,7695,165132671,164333375,18570,7744,167459085,166659789,19483,8125,173777444,172725651,99.43,99.37,98.61,98.60,94.77,94.71,95.03,95.14
1,00601,72,141,72141,105,49,2326414,2326414,18570,7744,167459085,166659789,33149,14192,298027589,294039825,0.57,0.63,1.39,1.40,0.32,0.35,0.78,0.79
2,00602,72,003,72003,41520,18073,83734431,79288158,41520,18073,83734431,79288158,41959,18258,117948080,79904246,100.00,100.00,100.00,100.00,98.95,98.99,70.99,99.23
3,00603,72,005,72005,54689,25653,82063867,81880442,54689,25653,82063867,81880442,60949,28430,195741178,94608641,100.00,100.00,100.00,100.00,89.73,90.23,41.92,86.55
4,00606,72,093,72093,6276,2740,94864349,94851862,6615,2877,109592548,109580061,6276,2740,94864349,94851862,94.88,95.24,86.56,86.56,100.00,100.00,100.00,100.00


In [16]:
print(ZiptoZcta_Crosswalk.info())
ZiptoZcta_Crosswalk.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41107 entries, 0 to 41106
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ZIP_CODE       41107 non-null  object
 1   PO_NAME        41107 non-null  object
 2   STATE          41107 non-null  object
 3   ZIP_TYPE       41107 non-null  object
 4   ZCTA           41107 non-null  object
 5   zip_join_type  41107 non-null  object
dtypes: object(6)
memory usage: 1.9+ MB
None


,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type
0,00501,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
1,00544,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
2,00601,Adjuntas,PR,Zip Code Area,00601,Zip matches ZCTA
3,00602,Aguada,PR,Zip Code Area,00602,Zip matches ZCTA
4,00603,Aguadilla,PR,Zip Code Area,00603,Zip matches ZCTA


In [17]:
print(all_geocodes.info())
all_geocodes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43838 entries, 0 to 43837
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   summary     43838 non-null  object
 1   state       43838 non-null  object
 2   county      43838 non-null  object
 3   county_sub  43838 non-null  object
 4   place       43838 non-null  object
 5   city        43838 non-null  object
 6   area        43838 non-null  object
dtypes: object(7)
memory usage: 2.3+ MB
None


,summary,state,county,county_sub,place,city,area
0,10,0,0,0,0,0,United States
1,40,1,0,0,0,0,Alabama
2,50,1,1,0,0,0,Autauga County
3,50,1,3,0,0,0,Baldwin County
4,50,1,5,0,0,0,Barbour County


In [18]:
all_geocodes['summary'] = all_geocodes['summary'].str.zfill(3)
all_geocodes['state'] = all_geocodes['state'].str.zfill(2)
all_geocodes['county'] = all_geocodes['county'].str.zfill(3)
all_geocodes['county_sub'] = all_geocodes['county_sub'].str.zfill(5)
all_geocodes['place'] = all_geocodes['place'].str.zfill(5)
all_geocodes['city'] = all_geocodes['city'].str.zfill(5)

In [19]:
all_geocodes.tail()

,summary,state,county,county_sub,place,city,area
43833,050,72,145,00000,00000,00000,Vega Baja Municipio
43834,050,72,147,00000,00000,00000,Vieques Municipio
43835,050,72,149,00000,00000,00000,Villalba Municipio
43836,050,72,151,00000,00000,00000,Yabucoa Municipio
43837,050,72,153,00000,00000,00000,Yauco Municipio


In [20]:
print(tier1geoid.info())
tier1geoid.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567 entries, 0 to 566
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Geoid   567 non-null    object
 1   State   567 non-null    object
 2   Top     567 non-null    object
 3   Popul   567 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 17.8+ KB
None


,Geoid,State,Top,Popul
0,04003,AZ,AZ-COCHISE,134489
1,04005,AZ,AZ-COCONINO,163882
2,04007,AZ,AZ-GILA,66230
3,04009,AZ,AZ-GRAHAM,49192
4,04011,AZ,AZ-GREENLEE,8446


In [21]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [22]:
# !pip install plotly

In [23]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [24]:
# %%timeit
# import plotly.express as px

# fig = px.choropleth(tier1geoid, geojson=counties, locations='Geoid', color='Popul',
#                            color_continuous_scale="Viridis",
#                            range_color=(0, 12),
#                            scope="usa",
#                            labels={'Popul':'Tier 1 Population'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [25]:
print(state_map.info())
state_map.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1004789 entries, 0 to 1004788
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   state_code  52 non-null     object
 1   state       52 non-null     object
 2   code        52 non-null     object
dtypes: object(3)
memory usage: 23.0+ MB
None


,state_code,state,code
0,01,Alabama,AL
1,02,Alaska,AK
2,04,Arizona,AZ
3,05,Arkansas,AR
4,06,California,CA


In [26]:
county_zcta['Geoid'] = county_zcta['county_geoid'].str[-5:]

In [27]:
tier1s = pd.merge(tier1geoid,county_zcta, on='Geoid', how='inner')

In [28]:
tier1s.head(20)

,Geoid,State,Top,Popul,state,county,zcta,zcta_geoid,county_geoid
0,04003,AZ,AZ-COCHISE,134489,4,003,85643,86000US85643,05000US04003
1,04003,AZ,AZ-COCHISE,134489,4,003,85620,86000US85620,05000US04003
2,04003,AZ,AZ-COCHISE,134489,4,003,85626,86000US85626,05000US04003
3,04003,AZ,AZ-COCHISE,134489,4,003,85615,86000US85615,05000US04003
4,04003,AZ,AZ-COCHISE,134489,4,003,85603,86000US85603,05000US04003
5,04003,AZ,AZ-COCHISE,134489,4,003,85608,86000US85608,05000US04003
6,04003,AZ,AZ-COCHISE,134489,4,003,85650,86000US85650,05000US04003
7,04003,AZ,AZ-COCHISE,134489,4,003,85607,86000US85607,05000US04003
8,04003,AZ,AZ-COCHISE,134489,4,003,85617,86000US85617,05000US04003
9,04003,AZ,AZ-COCHISE,134489,4,003,85613,86000US85613,05000US04003


In [29]:
tier1s = pd.merge(tier1s,ZiptoZcta_Crosswalk, left_on= ['zcta'], right_on = ['ZCTA'], how='inner')

In [30]:
tier1s.head(20)

,Geoid,State,Top,Popul,state,county,zcta,zcta_geoid,county_geoid,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,ZCTA,zip_join_type
0,04003,AZ,AZ-COCHISE,134489,4,003,85643,86000US85643,05000US04003,85643,Willcox,AZ,Zip Code Area,85643,Zip matches ZCTA
1,04003,AZ,AZ-COCHISE,134489,4,003,85643,86000US85643,05000US04003,85644,Willcox,AZ,Post Office or large volume customer,85643,Spatial join to ZCTA
2,04003,AZ,AZ-COCHISE,134489,4,003,85620,86000US85620,05000US04003,85620,Naco,AZ,Post Office or large volume customer,85620,Zip matches ZCTA
3,04003,AZ,AZ-COCHISE,134489,4,003,85626,86000US85626,05000US04003,85626,Pirtleville,AZ,Post Office or large volume customer,85626,Zip matches ZCTA
4,04003,AZ,AZ-COCHISE,134489,4,003,85615,86000US85615,05000US04003,85615,Hereford,AZ,Zip Code Area,85615,Zip matches ZCTA
5,04003,AZ,AZ-COCHISE,134489,4,003,85603,86000US85603,05000US04003,85603,Bisbee,AZ,Zip Code Area,85603,Zip matches ZCTA
6,04003,AZ,AZ-COCHISE,134489,4,003,85608,86000US85608,05000US04003,85608,Douglas,AZ,Post Office or large volume customer,85608,Zip matches ZCTA
7,04003,AZ,AZ-COCHISE,134489,4,003,85650,86000US85650,05000US04003,85650,Sierra Vista,AZ,Zip Code Area,85650,Zip matches ZCTA
8,04003,AZ,AZ-COCHISE,134489,4,003,85607,86000US85607,05000US04003,85607,Douglas,AZ,Zip Code Area,85607,Zip matches ZCTA
9,04003,AZ,AZ-COCHISE,134489,4,003,85607,86000US85607,05000US04003,85655,Douglas,AZ,Post Office or large volume customer,85607,Spatial join to ZCTA


In [31]:
tier1s = pd.merge(tier1s,state_map, left_on= ['STATE'], right_on = ['code'], how='inner')

In [32]:
tier1s = tier1s.filter(["Geoid","State","Top","Popul","zcta","zcta_geoid","county_geoid","ZIP_CODE","PO_NAME","ZIP_TYPE","zip_join_type","code"])

In [33]:
tier1s.head(20)

,Geoid,State,Top,Popul,zcta,zcta_geoid,county_geoid,ZIP_CODE,PO_NAME,ZIP_TYPE,zip_join_type,code
0,04003,AZ,AZ-COCHISE,134489,85643,86000US85643,05000US04003,85643,Willcox,Zip Code Area,Zip matches ZCTA,AZ
1,04003,AZ,AZ-COCHISE,134489,85643,86000US85643,05000US04003,85644,Willcox,Post Office or large volume customer,Spatial join to ZCTA,AZ
2,04003,AZ,AZ-COCHISE,134489,85620,86000US85620,05000US04003,85620,Naco,Post Office or large volume customer,Zip matches ZCTA,AZ
3,04003,AZ,AZ-COCHISE,134489,85626,86000US85626,05000US04003,85626,Pirtleville,Post Office or large volume customer,Zip matches ZCTA,AZ
4,04003,AZ,AZ-COCHISE,134489,85615,86000US85615,05000US04003,85615,Hereford,Zip Code Area,Zip matches ZCTA,AZ
5,04003,AZ,AZ-COCHISE,134489,85603,86000US85603,05000US04003,85603,Bisbee,Zip Code Area,Zip matches ZCTA,AZ
6,04003,AZ,AZ-COCHISE,134489,85608,86000US85608,05000US04003,85608,Douglas,Post Office or large volume customer,Zip matches ZCTA,AZ
7,04003,AZ,AZ-COCHISE,134489,85650,86000US85650,05000US04003,85650,Sierra Vista,Zip Code Area,Zip matches ZCTA,AZ
8,04003,AZ,AZ-COCHISE,134489,85607,86000US85607,05000US04003,85607,Douglas,Zip Code Area,Zip matches ZCTA,AZ
9,04003,AZ,AZ-COCHISE,134489,85607,86000US85607,05000US04003,85655,Douglas,Post Office or large volume customer,Spatial join to ZCTA,AZ


In [34]:
tier1s.to_csv('s3://hc-data-temp/tier1s.csv', index=False, header=True)

In [35]:
tier1s.to_csv('/Users/dletendr/Downloads/tier1s.csv', index=False, header=True)

In [36]:
tier1s['loc_key'] = tier1s['code'] + '-' + tier1s['PO_NAME'].str.upper()

In [37]:
area_codes = pd.merge(area_codes,state_map, left_on= ['state'], right_on = ['state'], how='inner')

In [38]:
area_codes.head()

,areacode,city,state,country,lat,long,state_code,code
0,201,Bayonne,New Jersey,US,40.66871,-74.11431,34,NJ
1,201,Bergenfield,New Jersey,US,40.92760,-73.99736,34,NJ
2,201,Cliffside Park,New Jersey,US,40.82149,-73.98764,34,NJ
3,201,Englewood,New Jersey,US,40.89288,-73.97264,34,NJ
4,201,Fair Lawn,New Jersey,US,40.94038,-74.13181,34,NJ


In [39]:
area_codes['loc_key'] = area_codes['code'] + '-' + area_codes['city'].str.upper()

In [40]:
tier1s = pd.merge(tier1s,area_codes, left_on= ['loc_key'], right_on = ['loc_key'], how='left')

In [41]:
tier1s.head(20)

,Geoid,State,Top,Popul,zcta,zcta_geoid,county_geoid,ZIP_CODE,PO_NAME,ZIP_TYPE,zip_join_type,code_x,loc_key,areacode,city,state,country,lat,long,state_code,code_y
0,04003,AZ,AZ-COCHISE,134489,85643,86000US85643,05000US04003,85643,Willcox,Zip Code Area,Zip matches ZCTA,AZ,AZ-WILLCOX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,04003,AZ,AZ-COCHISE,134489,85643,86000US85643,05000US04003,85644,Willcox,Post Office or large volume customer,Spatial join to ZCTA,AZ,AZ-WILLCOX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04003,AZ,AZ-COCHISE,134489,85620,86000US85620,05000US04003,85620,Naco,Post Office or large volume customer,Zip matches ZCTA,AZ,AZ-NACO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04003,AZ,AZ-COCHISE,134489,85626,86000US85626,05000US04003,85626,Pirtleville,Post Office or large volume customer,Zip matches ZCTA,AZ,AZ-PIRTLEVILLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,04003,AZ,AZ-COCHISE,134489,85615,86000US85615,05000US04003,85615,Hereford,Zip Code Area,Zip matches ZCTA,AZ,AZ-HEREFORD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,04003,AZ,AZ-COCHISE,134489,85603,86000US85603,05000US04003,85603,Bisbee,Zip Code Area,Zip matches ZCTA,AZ,AZ-BISBEE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,04003,AZ,AZ-COCHISE,134489,85608,86000US85608,05000US04003,85608,Douglas,Post Office or large volume customer,Zip matches ZCTA,AZ,AZ-DOUGLAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,04003,AZ,AZ-COCHISE,134489,85650,86000US85650,05000US04003,85650,Sierra Vista,Zip Code Area,Zip matches ZCTA,AZ,AZ-SIERRA VISTA,520,Sierra Vista,Arizona,US,31.55454,-110.30369,04,AZ
8,04003,AZ,AZ-COCHISE,134489,85607,86000US85607,05000US04003,85607,Douglas,Zip Code Area,Zip matches ZCTA,AZ,AZ-DOUGLAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,04003,AZ,AZ-COCHISE,134489,85607,86000US85607,05000US04003,85655,Douglas,Post Office or large volume customer,Spatial join to ZCTA,AZ,AZ-DOUGLAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
area_code_check = pd.merge(area_codes, tier1s, left_on= ['loc_key'], right_on = ['loc_key'], how='left')

In [43]:
area_code_check.head()

,areacode_x,city_x,state_x,country_x,lat_x,long_x,state_code_x,code,loc_key,Geoid,State,Top,Popul,zcta,zcta_geoid,county_geoid,ZIP_CODE,PO_NAME,ZIP_TYPE,zip_join_type,code_x,areacode_y,city_y,state_y,country_y,lat_y,long_y,state_code_y,code_y
0,201,Bayonne,New Jersey,US,40.66871,-74.11431,34,NJ,NJ-BAYONNE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201,Bergenfield,New Jersey,US,40.92760,-73.99736,34,NJ,NJ-BERGENFIELD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201,Cliffside Park,New Jersey,US,40.82149,-73.98764,34,NJ,NJ-CLIFFSIDE PARK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201,Englewood,New Jersey,US,40.89288,-73.97264,34,NJ,NJ-ENGLEWOOD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201,Fair Lawn,New Jersey,US,40.94038,-74.13181,34,NJ,NJ-FAIR LAWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
area_code_check.to_csv('s3://hc-data-temp/area_code_check.csv', index=False, header=True)

In [45]:
area_code_targets = tier1s.groupby(by=["Geoid","areacode","loc_key"],as_index=False).count()

In [46]:
area_code_targets = area_code_targets.filter(["Geoid","areacode","loc_key"])

In [47]:
# area_code_targets.to_csv('s3://hc-data-temp/area_code_targets.csv', index=False, header=True)

In [48]:
area_code_targets.to_csv('/Users/dletendr/Downloads/area_code_targets.csv', index=False, header=True)